In [54]:

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.motion_refiner_4D failed: Traceback (most recent call last):
  File "/home/arthur/.conda/envs/py38_cu11_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/arthur/.conda/envs/py38_cu11_2/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/arthur/.conda/envs/py38_cu11_2/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/arthur/catkin_ws/src/NL_trajectory_reshaper/src/motion_refiner_4D.py", line 14, in <module>
    from config import *
ModuleNotFoundError: No module named 'config'
]


In [55]:
import numpy as np
from src.motion_refiner_4D import Motion_refiner
from src.config import *
# base_folder = "/home/tum/data/"
# base_folder = "/home/arthur/local_data/" 

traj_n = 40
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=True)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = np.concatenate([feature_indices,obj_sim_indices, obj_poses_indices])

loading CLIP model... done


In [56]:
dataset_name = "4D_100k_scaping_factor"

X,Y, data = mr.load_dataset(dataset_name, filter_data = True, base_path=base_folder+"data/")
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)


loading dataset:  4D_100k_scaping_factor ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (99092, 953) 	Y: (99092, 160)
ATTENTION!!! dataset and MR indexes dont match!!!
motion refiner final index =  697
dataset final index =  953
Train X: (69363, 953) 	Y: (69363, 160)
Test  X: (19819, 953) 	Y: (19819, 160)
Val   X: (9910, 953) 	Y: (9910, 160)


In [43]:
from tqdm import tqdm
change_img_base=['/mnt/tumdata/image_dataset',image_dataset_folder]

all_image_paths = []
for i, d in tqdm(enumerate(data)):
    image_paths = d["image_paths"]

    if not change_img_base is None:
        for ti in range(len(image_paths)):
            all_image_paths.append(image_paths[ti].replace(change_img_base[0], change_img_base[1]))
    
print(len(all_image_paths))
set_res = set(all_image_paths) 
print("The unique elements of the input list using set():\n") 
all_image_paths = (list(set_res))
print(len(all_image_paths))

# for item in list_res: 
#     print(item) 
# img_paths = [dd["image_paths"] for d in data]

99092it [00:00, 687159.06it/s]

395345
The unique elements of the input list using set():

9290


In [66]:
import json
obj_library = {}
with open(image_dataset_folder+"imagenet1000_clsidx_to_labels.txt") as f:
    obj_library = json.load(f)

obj_names = []
for k,v in obj_library.items():
    obj_names= obj_names+obj_library[k] 
print(len(obj_names))

1858


In [70]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm
from PIL import Image


# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)


# def load_image(img_paths):
#     img = 
#     return img

def image_loader(lst, n):
    for i in range(0, len(lst), n):
        b = lst[i:i + n]
        classes = [c.split("/")[-2] for c in b]
        images = torch.concat([preprocess(Image.open(im)).unsqueeze(0) for im in b])
        yield classes,images
def get_image_features(image_paths):
    all_features = []

    with torch.no_grad():
        for classes, images in image_loader(image_paths,100):
            print(images.shape)
            features = model.encode_image(images.to(device))
            all_features.append(features)
    return torch.cat(all_features).cpu().numpy()

def text_loader(lst, n):
    for i in range(0, len(lst), n):
        yield clip.tokenize(lst[i:i + n]).to(device)
def get_text_features(text):
    all_features = []

    with torch.no_grad():
        for t in text_loader(text,100):
            features = model.encode_text(t.to(device))
            all_features.append(features)
    return torch.cat(all_features).cpu().numpy()

all_obj_names_features = get_text_features(obj_names)

# all_features = get_image_features(all_image_paths)
# all_features.shape


In [72]:
import json
obj_names_clip_emb = {}
for f, p in zip(all_obj_names_features,obj_names):
    obj_names_clip_emb[p] = f.tolist()

with open(image_dataset_folder+'obj_names_clip_emb.json', 'w', encoding='utf-8') as f:
    json.dump(obj_names_clip_emb, f, ensure_ascii=False, indent=4)

In [50]:
import json
images_clip_emb = {}
for f, p in zip(all_features,all_image_paths):
    images_clip_emb[p] = f.tolist()
    
with open(image_dataset_folder+'images_clip_emb.json', 'w', encoding='utf-8') as f:
    json.dump(images_clip_emb, f, ensure_ascii=False, indent=4)

In [80]:
from src.motion_refiner_4D import Motion_refiner

mr = Motion_refiner(traj_n = 40, clip_only=True, locality_factor=True, poses_on_features=True, load_precomp_emb=True)


TypeError: __init__() got an unexpected keyword argument 'load_precomp_emb'

In [ ]:
X_,Y_ = mr.prepare_data(data,deltas=False, change_img_base=['/mnt/tumdata/image_dataset',args.image_dataset_dir])


In [53]:
images_clip_emb2={}
with open(image_dataset_folder+"images_clip_emb.json") as f:
    images_clip_emb2 = json.load(f)
print(len(images_clip_emb2.keys()))

9290


In [8]:
X,Y = mr.prepare_data(data[:5],deltas=False, change_img_base=['/mnt/tumdata/image_dataset',image_dataset_folder])

text_features  (5, 512)


1it [00:00,  4.25it/s]

text_clip_features  torch.Size([1, 512])


2it [00:38, 22.51s/it]

text_clip_features  torch.Size([1, 512])


2it [01:03, 31.55s/it]


KeyboardInterrupt: 

In [4]:
# # %matplotlib qt
# from src.functions import *
# idx = random.choices(range(len(data)),k=3)
# data_sample = list(np.array(data)[idx])
# show_data4D(data_sample)
# # print(idx)

## Dynamic time warping (DTW)

In [6]:
from src.functions import *

n = -1
trajs_x = mr.prepare_x(X_valid)[:n,6:,:]
trajs_y = list_to_wp_seq(y_valid,d=4)[:n,:,:]


In [7]:
import similaritymeasures
def compute_metrics(trajs_x, trajs_y, filter_nan=False):
    """Computes the similarity metrics between 2 trajectories:
    returns: dict(pcm,df,area,cl,dtw,mae,mse), metrics over each sample"""

    metrics = {"pcm":None,"dfd":None,"area":None,"cl":None,"dtw":None,"mae":None,"mse":None}

    metrics_h = np.zeros((trajs_x.shape[0],7))
    for i,(exp_data, num_data) in enumerate(zip(trajs_x, trajs_y)):

        pcm = similaritymeasures.pcm(exp_data, num_data) # Partial Curve Mapping
        dfd = similaritymeasures.frechet_dist(exp_data, num_data) # Discrete Frechet distance
        area = similaritymeasures.area_between_two_curves(exp_data, num_data) # area between two curves
        cl = similaritymeasures.curve_length_measure(exp_data, num_data)# Curve Length based similarity measure
        dtw, d = similaritymeasures.dtw(exp_data, num_data) # Dynamic Time Warping distance
        mae = similaritymeasures.mae(exp_data, num_data) # mean absolute error
        mse = similaritymeasures.mse(exp_data, num_data) # mean squared error

        metrics_h[i,:] = [pcm,dfd,area,cl,dtw,mae,mse]

    if filter_nan:
        metrics_h = metrics_h[~np.isnan(metrics_h).any(axis=1),:]
    metrics_v = np.mean(metrics_h,axis=0)
    for k,v in zip(metrics.keys(), metrics_v):
        metrics[k]=v
        print(k+":\t",v)
    return metrics, metrics_h

metrics, metrics_h = compute_metrics(trajs_x, trajs_y)

pcm:	 nan
dfd:	 0.14597155479503446
area:	 0.027609880103219818
cl:	 nan
dtw:	 3.7486616869839575
mae:	 0.0276043962328285
mse:	 0.005053778124616486
